# Palm Tree Detection with YOLOv8
## 1. Install Dependencies

In [1]:
# !pip install ultralytics
# !pip install pandas

## 2. Import Libraries

In [2]:
import pandas as pd
import os
from tqdm import tqdm
import shutil

## 3. Convert CSV to YOLO Format

In [3]:
def convert_csv_to_yolo(csv_file, output_dir):
    # Read CSV file
    df = pd.read_csv(csv_file)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Group by filename
    for filename, group in tqdm(df.groupby('filename')):
        yolo_anns = []
        width = group['width'].iloc[0]
        height = group['height'].iloc[0]
        
        # Convert each bbox to YOLO format
        for _, row in group.iterrows():
            # Convert to normalized coordinates
            x_center = ((row['xmin'] + row['xmax']) / 2) / width
            y_center = ((row['ymin'] + row['ymax']) / 2) / height
            w = (row['xmax'] - row['xmin']) / width
            h = (row['ymax'] - row['ymin']) / height
            
            # Class ID (0 for palm tree)
            yolo_anns.append(f"0 {x_center} {y_center} {w} {h}")
        
        # Save annotations
        base_name = os.path.splitext(filename)[0]
        with open(os.path.join(output_dir, f"{base_name}.txt"), 'w') as f:
            f.write('\n'.join(yolo_anns))

In [4]:
# Convert train and test annotations
train_csv = 'dataset/Palm-Counting-349images/train_labels.csv'
test_csv = 'dataset/Palm-Counting-349images/test_labels.csv'
train_output = 'dataset/yolo/labels/train'
test_output = 'dataset/yolo/labels/test'

# Create directories
os.makedirs('dataset/yolo/images/train', exist_ok=True)
os.makedirs('dataset/yolo/images/test', exist_ok=True)

# Convert annotations
convert_csv_to_yolo(train_csv, train_output)
convert_csv_to_yolo(test_csv, test_output)

# Copy images
for img in os.listdir('dataset/Palm-Counting-349images/train/images'):
    shutil.copy2(
        os.path.join('dataset/Palm-Counting-349images/train/images', img),
        os.path.join('dataset/yolo/images/train', img)
    )

for img in os.listdir('dataset/Palm-Counting-349images/test/images'):
    shutil.copy2(
        os.path.join('dataset/Palm-Counting-349images/test/images', img),
        os.path.join('dataset/yolo/images/test', img)
    )

100%|██████████| 70/70 [00:00<00:00, 1106.63it/s]


## 3.1 Create Validation Split

In [5]:
import numpy as np

# Create validation directory
os.makedirs('dataset/yolo/images/val', exist_ok=True)
os.makedirs('dataset/yolo/labels/val', exist_ok=True)

# Read train CSV
train_df = pd.read_csv(train_csv)

# Get unique filenames
unique_files = train_df['filename'].unique()

# Random split - 80% train, 20% validation
np.random.seed(42)
val_files = np.random.choice(unique_files, size=int(len(unique_files) * 0.2), replace=False)
val_files = set(val_files)

# Move validation files
for img in os.listdir('dataset/yolo/images/train'):
    if img in val_files:
        # Move image
        shutil.move(
            os.path.join('dataset/yolo/images/train', img),
            os.path.join('dataset/yolo/images/val', img)
        )
        # Move label file
        label_file = os.path.splitext(img)[0] + '.txt'
        if os.path.exists(os.path.join('dataset/yolo/labels/train', label_file)):
            shutil.move(
                os.path.join('dataset/yolo/labels/train', label_file),
                os.path.join('dataset/yolo/labels/val', label_file)
            )

## 4. Create YAML Configuration

In [6]:
# # Update YAML config to include validation split
# yaml_content = """
# path: dataset/yolo  # dataset root dir
# train: images/train  # train images (80%)
# val: images/val  # validation images (20%)
# test: images/test  # test images

# # Classes
# names:
#   0: palm_tree
# """

# with open('dataset/yolo/data.yaml', 'w') as f:
#     f.write(yaml_content)

## 5. Train YOLOv8 Model

In [7]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load pretrained model

# Train the model
results = model.train(
    data='dataset/yolo/data.yaml',
    epochs=100,
    imgsz=640,
    batch=32,
    name='palm_tree_detection'
)

New https://pypi.org/project/ultralytics/8.3.93 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16064MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/yolo/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=palm_tree_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

train: Scanning /home/imran4070/Downloads/datasets/kaggle/dataset/yolo/labels/train.cache... 224 images, 0 backgrounds, 0 corrupt: 100%|██████████| 224/224 [00:00<?, ?it/s]
val: Scanning /home/imran4070/Downloads/datasets/kaggle/dataset/yolo/labels/val.cache... 55 images, 0 backgrounds, 0 corrupt: 100%|██████████| 55/55 [00:00<?, ?it/s]


Plotting labels to runs/detect/palm_tree_detection/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/palm_tree_detection
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      6.92G      2.562      3.547      2.011       1313        640: 100%|██████████| 7/7 [00:24<00:00,  3.44s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:12<00:00, 12.87s/it]

                   all         55       2104     0.0639      0.501     0.0486     0.0209


...

    100/100      4.88G      1.016     0.4598     0.9906        976        640: 100%|██████████| 7/7 [00:03<00:00,  2.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  9.35it/s]

                   all         55       2104      0.893      0.881      0.929      0.576



100 epochs completed in 0.104 hours.
Optimizer stripped from runs/detect/palm_tree_detection/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/palm_tree_detection/weights/best.pt, 6.2MB

Validating runs/detect/palm_tree_detection/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16064MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.10it/s]


                   all         55       2104      0.888      0.891      0.932      0.587
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/palm_tree_detection


## 6. Validate Model

In [10]:
# Validate the model
metrics = model.val()

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16064MiB)


val: Scanning /home/imran4070/Downloads/datasets/kaggle/dataset/yolo/labels/val.cache... 55 images, 0 backgrounds, 0 corrupt: 100%|██████████| 55/55 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.89s/it]


                   all         55       2104      0.888      0.891      0.932      0.588
Speed: 0.7ms preprocess, 1.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/palm_tree_detection3
